In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

import torch
import random 
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/kaggle/input/mushroom-classification/mushrooms.csv")
df_2 = df.copy()

In [ ]:
df.head()

In [ ]:
ax = df['class'].value_counts().plot.bar(color= ['darkblue', 'lightblue']);

plt.title('Quantity of each class in the dataset', fontsize = 15)
plt.xticks(rotation=0)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
for i in ax.patches:
    ax.text(i.get_x() + 0.12, i.get_height()-300, i.get_height(), fontsize=16,
                color='white')
    
plt.show()

In [ ]:
df.apply(lambda x: x.nunique())

In [ ]:
df = df.drop(['veil-type'], axis=1)
y = df['class']
X = df.drop(['class'], axis=1)

In [ ]:
y = y.apply(lambda x : 1 if x == 'e' else 0)

# EXPIRIMENT 1. LABEL ENCODING

In [ ]:
l_t = LabelEncoder()
X_label_encoding = X.apply(lambda x : l_t.fit_transform(x))

In [ ]:
X_label_encoding.head()

In [ ]:
X_label_encoding.shape

In [ ]:
torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# torch.cuda.get_device_name(0)

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(X_label_encoding, y, test_size = 0.2, shuffle= True)

In [ ]:
X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, first_neurons, n_hidden_neurons):
        super().__init__()
        self.fc1 = nn.Linear(first_neurons, n_hidden_neurons)
        self.activ1 = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.activ2 = nn.ReLU()
        self.fc3 = nn.Linear(n_hidden_neurons, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
      
        x = self.fc2(x)
        x = self.activ2(x)
        
        x = self.fc3(x)
        return x 

In [ ]:
myNet = NeuralNetwork(21, 50)
myNet = myNet.to(device)
myNet.parameters()
# list(myNet.parameters())
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(myNet.parameters(),  lr = 10 **(-4))

In [ ]:
X_test = X_test.to(device)
y_test = y_test.to(device)

In [ ]:
acc = []
losss = []
def training(model, batch_size, epochs, loss, optimizer):
    for epoch in range(epochs):
        order = np.random.permutation(len(X_train))
        for start_ind in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            batch_indexes = order[start_ind:start_ind + batch_size]
            X_batch = X_train[batch_indexes].to(device)
            y_batch = y_train[batch_indexes].to(device)

            preds = model.forward(X_batch)
            loss_value = loss(preds, y_batch.long())
            loss_value.backward()
            optimizer.step()

        test_preds = model.forward(X_test)
        losss.append(loss(test_preds, y_test.long()).to('cpu').item())

        accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().to('cpu')
        acc.append(accuracy)
        if (epoch + 1) % 5 == 0:
            print(f'epoch: {epoch + 1}, acc:{accuracy}')

In [ ]:
epoch = 100
batch_size = 200
training(myNet, batch_size, epoch, loss, optimizer)

In [ ]:
x = np.arange(1,101)
y_v = np.array(acc) * 100
max_value = y_v.max().reshape(-1)
y_values = np.array(acc) * 100
y_ticks =   np.concatenate((np.arange(y_v[1:].min(), y_v.max(), 5), max_value),axis = 0)
plt.plot(x, y_v,  color ='darkblue', linewidth=4)
plt.yticks(y_ticks)
plt.grid(axis='y', linestyle='-')
plt.grid(axis='x', linestyle='-')
plt.title('Accuracy', fontsize = 15 );
plt.xlabel('epoch', fontsize=12);
plt.show();

# EXPIRIMENT 2. ONE HOT ENCODING

In [ ]:
X_one_hot_encoding  =  pd.get_dummies(X)

In [ ]:
X_one_hot_encoding.shape

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(X_one_hot_encoding, y, test_size = 0.2, shuffle= True)
X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)

In [ ]:
myNet_OHE = NeuralNetwork(116, 50)
myNet_OHE = myNet_OHE.to(device)

In [ ]:
# list(myNet_OHE.parameters())

In [ ]:
X_test = X_test.to(device)
y_test = y_test.to(device)
acc = []

In [ ]:
epoch = 100
batch_size = 200
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(myNet_OHE.parameters(),  lr = 10 **(-4))
training(myNet_OHE, batch_size, epoch, loss, optimizer)

In [ ]:
x = np.arange(1,101)
y_label_encoding = y_v
y_one_hot_encoding = np.array(acc) * 100
max_value_1 = y_label_encoding.max().reshape(-1)
max_value_2 = y_one_hot_encoding.max().reshape(-1)
y_values = np.array(acc) * 100
y_ticks =   np.concatenate((np.arange(y_v[1:].min(), y_v.max(), 5), max_value_1,
                            max_value_2,),axis = 0)
plt.plot(x, y_label_encoding,  color ='darkblue', linewidth=3, label ='LabelEncoding')
plt.plot(x, y_one_hot_encoding,  color ='red', linewidth=3, label = 'OneHotEncoding')
plt.yticks(y_ticks)
plt.grid(axis='y', linestyle='-')
plt.grid(axis='x', linestyle='-')
plt.title('Accuracy', fontsize = 15 );
plt.xlabel('epoch', fontsize=12);
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show();